# 사용자 선호에 맞는 시 창작 모델

### 0. 환경 설정

In [1]:
!python -m pip install --upgrade pip

In [2]:
!pip install typing-extensions pydantic openai

In [ ]:
# !pip install typing_extensions

In [4]:
!pip install datasets transformers peft trl bitsandbytes

In [3]:
import os
import torch

os.environ["WANDB_DISABLED"] = "true"                     # 모델 학습 추적 비활성화
os.environ["TOKENIZERS_PARALLELISM"] = "false"            # 토크나이저 병렬 처리 비활성화

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 설정 변수 : 사용 가능한 장치 선택


### 1. 지도학습 (기반모델 Q-LoRA 파인튜닝)

(1) 학습용 데이터 준비

In [4]:
import json
from datasets import Dataset

# 데이터 로드 및 데이터셋 변환
dataset_path = './korean_poetry_dataset.json'

with open("./korean_poetry_dataset.json", "r", encoding="utf-8") as f:
    poem_data = json.load(f)

processed_data = [{'topic': item['text']['topic'], 'poem': item['text']['poem']} for item in poem_data]

train_dataset = Dataset.from_list(processed_data)

In [5]:
from transformers import AutoTokenizer

model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 데이터 전처라 함수 (토큰화 + Labels 추가)
def preprocessed_text(sample):
    input_texts = [f"topic: {t}\npoem: {p}" for t, p in zip(sample['topic'], sample['poem'])]

    model_inputs = tokenizer(
        input_texts,
        padding='max_length',
        max_length=512,
        truncation=True
    )
    model_inputs['labels'] = model_inputs['input_ids'].copy()
    pad_token_id = tokenizer.pad_token_id
    model_inputs['labels'] = [
        [(l if l != pad_token_id else -100) for l in label] for label in model_inputs['labels']
    ]

    return model_inputs

In [ ]:
# 데이터 셋 변환
train_dataset = train_dataset.map(
    preprocessed_text,
    batched=True,
    remove_columns=["topic", "poem"]
)

# 데이터 셋 확인
train_dataset[0]

Map:   0%|          | 0/2600 [00:00<?, ? examples/s]

{'input_ids': [128000,
  16816,
  25,
  108466,
  49531,
  198,
  5481,
  336,
  25,
  108466,
  49531,
  34804,
  62398,
  119873,
  21028,
  103213,
  107335,
  11,
  2355,
  102837,
  49085,
  81673,
  110578,
  13094,
  125693,
  107054,
  123637,
  11,
  2355,
  41381,
  104374,
  109580,
  74177,
  54542,
  53400,
  119873,
  21028,
  101003,
  29102,
  107335,
  11,
  31879,
  123849,
  22035,
  21121,
  105164,
  66965,
  21028,
  122352,
  41953,
  105220,
  19954,
  11,
  2355,
  101314,
  29102,
  106646,
  105250,
  103843,
  17835,
  101266,
  116129,
  124742,
  13094,
  2355,
  54059,
  102436,
  49085,
  124301,
  86503,
  126690,
  112795,
  13,
  31879,
  26799,
  11,
  113857,
  16969,
  23955,
  121385,
  119873,
  18359,
  75086,
  41381,
  35495,
  11,
  2355,
  101532,
  20565,
  94772,
  39250,
  108922,
  18359,
  67236,
  119443,
  101203,
  11,
  2355,
  110955,
  109723,
  116548,
  105642,
  23955,
  21028,
  87134,
  43139,
  31879,
  108307,
  114149,
  1

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

### 2. 파인튜닝 학습 준비

- 양자화 설정 > 모델 로드
- 학습 모드로 전환
- LoRA 학습 설정
- TrainingArguments 설정

In [9]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [10]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model.gradient_checkpointing_enable()
model.config.use_cache = False
model.config.attn_implementation = 'flash_attention_2'

Exception in thread Thread-46 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\Playdata\anaconda3\envs\llm_env\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\Playdata\anaconda3\envs\llm_env\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Playdata\anaconda3\envs\llm_env\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 6: invalid start byte


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
# 양자화모델을 훈련하기 위한 준비
from peft import get_peft_model

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# 모델 학습 모드 설정
model.train()

trainable params: 4,587,520 || all params: 3,217,337,344 || trainable%: 0.1426


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Lin

In [13]:
# TrainingArguments 설정
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./q_lora_poem",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    optim="adamw_bnb_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_308\147409695.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


(3) 학습 진행

In [ ]:
trainer.train()

### 2. 학습된 모델로 시(응답) 생성

(1) 모델 로드

In [14]:
from transformers import pipeline

qlora_checkpoint = './q_lora_poem/checkpoint-975'

model = AutoModelForCausalLM.from_pretrained(qlora_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generate_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    batch_size=2
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [15]:
topics = ["바람", "비", "노을", "달빛", "안개", "사랑", "이별", "운명", "기다림", "후회", "추억", "시간", "청춘", "변화", "마지막 순간", "군중", "밤거리", "버스", "인생", "빌딩", "사람들", "거짓말", "욕망", "돈", "권력", "비밀", "죽음", "희망", "동물", "자연", "도시", "바다", "산", "하늘", "별", "꽃", "나무", "강", "바위", "흙", "눈", "빗방울", "눈물", "웃음"]

eval_file = 'rlhf_eval_data.json'

try:
    with open(eval_file, 'r', encoding='utf-8') as f:
        eval_dataset = json.load(f)
except FileNotFoundError:
    eval_dataset = []

In [16]:
num_batches = 5
batch_size = 20
total_samples = num_batches * batch_size
generated_samples = len(eval_dataset)

(2) 시 생성

In [18]:
import time
import random
from tqdm import tqdm

def generate_poem_batch():
    batch_data = []

    with tqdm(total=batch_size, desc='<시 생성 중>', leave=False) as t:
        for _ in range(batch_size):
            topic = random.choice(topics)
            input_text = f"topic: {topic}\npoem:"

            start_time = time.time()
            poem = generate_pipeline(
                input_text,
                max_new_tokens=100,
                top_p=0.9,
                # top_k=50,
                temperature=0.8
            )[0]['generated_text']
            end_time = time.time()

            gen_time = end_time - start_time

            batch_data.append({
                'topic': topic,
                'poem': poem,
                'selected': None
            })

            t.update(1)
            
            global generated_samples
            generated_samples += 1
            complete_rate = (generated_samples / total_samples) * 100
            remaining_time = ((total_samples - generated_samples) * gen_time) / 60

            print(f"\n{generated_samples}/{total_samples}개 완료 | ({complete_rate:.2f}%) | 예상 완료 시간: {remaining_time:.2f}분")
            print('-' * 100)

    return batch_data

In [21]:
for _ in tqdm(range(num_batches), desc='<전체 진행 상황>', position=0, leave=False):
    eval_dataset.extend(generate_poem_batch())

    with open(eval_file, 'w', encoding='utf-8') as f:
        json.dump(eval_dataset, f, ensure_ascii=False, indent=4)

<전체 진행 상황>:   0%|          | 0/5 [00:00<?, ?it/s]


1/100개 완료 | (1.00%) | 예상 완료 시간: 175.03분
----------------------------------------------------------------------------------------------------



2/100개 완료 | (2.00%) | 예상 완료 시간: 269.58분
----------------------------------------------------------------------------------------------------



3/100개 완료 | (3.00%) | 예상 완료 시간: 270.02분
----------------------------------------------------------------------------------------------------



4/100개 완료 | (4.00%) | 예상 완료 시간: 267.78분
----------------------------------------------------------------------------------------------------


...


96/100개 완료 | (96.00%) | 예상 완료 시간: 6.38분
----------------------------------------------------------------------------------------------------



97/100개 완료 | (97.00%) | 예상 완료 시간: 4.97분
----------------------------------------------------------------------------------------------------



98/100개 완료 | (98.00%) | 예상 완료 시간: 5.33분
----------------------------------------------------------------------------------------------------



99/100개 완료 | (99.00%) | 예상 완료 시간: 1.92분
----------------------------------------------------------------------------------------------------



100/100개 완료 | (100.00%) | 예상 완료 시간: 0.00분
----------------------------------------------------------------------------------------------------


(3) 피드백

- 생성된 시에 대해 selected = true 로 수정해 피드백 반영

### 3. Reward Model 학습

(1) 데이터 로드 및 처리

In [26]:
with open(eval_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

    reward_data = [
        {'text_a': f"주제: {item['topic']}", 'text_b': item['poem']} for item in evaluation_data if item['selected']
    ]

    reward_dataset = Dataset.from_list(reward_data)

In [27]:
# 데이터 전처리 함수 추가 (batch 처리 가능)
def preprocessed_reward_text(sample):
    model_inputs = tokenizer(
        sample['text_a'],
        text_pair=sample['text_b'],
        padding='max_length',
        max_length=512,
        truncation=True
    )
    model_inputs['labels'] = model_inputs['input_ids'].copy()
    pad_token_id = tokenizer.pad_token_id
    model_inputs['labels'] = [
        [(l if l != pad_token_id else -100) for l in label] for label in model_inputs['labels']
    ]

    return model_inputs

In [28]:
tokenizer.pad_token = tokenizer.eos_token

reward_dataset = reward_dataset.map(
    preprocessed_reward_text,
    batched=True,
    remove_columns=['text_a', 'text_b']
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

(2)

- 양자화 설정 > 모델 로드
- LoRA 학습 설정
- TrainingArguments 설정

In [29]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    device_map="auto"
)

In [30]:
from peft import get_peft_model, prepare_model_for_kbit_training

reward_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

reward_model = prepare_model_for_kbit_training(reward_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
reward_model = get_peft_model(reward_model, lora_config)

In [ ]:
reward_training_args = TrainingArguments(
    output_dir="./reward_model",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=True
)

reward_trainer = Trainer(
    model=reward_model,
    args=reward_training_args,
    train_dataset=reward_dataset,
    tokenizer=tokenizer
)

### 4. RLHF (ORPO)

(1) 모델 로드

In [ ]:
model = AutoModelForCausalLM.from_pretrained(qlora_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.train()
model.cuda()

for param in model.parameters():
    param.requires_grad = True

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

(2) ORPO 데이터셋 준비

In [ ]:
with open(eval_file, "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

orpo_data = []

for item in evaluation_data:
    if item['selected']:
        prompt_text = f'주제: {item["topic"]}\n이 주제에 맞는 시를 작성해 주세요.'
        chosen_text = item['poem']
        rejected_text = ""

        tokenized_prompt = tokenizer(prompt_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")
        tokenized_chosen = tokenizer(chosen_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")
        tokenized_rejected = tokenizer(rejected_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")

        orpo_data.append({
            "prompt": prompt_text,
            "chosen": chosen_text,
            "rejected": rejected_text,
            "prompt_input_ids": tokenized_prompt['input_ids'].squeeze(0).cuda(),
            "prompt_attention_mask": tokenized_prompt['attention_mask'].squeeze(0).cuda(),
            "chosen_input_ids": tokenized_chosen['input_ids'].squeeze(0).cuda(),
            "chosen_attention_mask": tokenized_chosen['attention_mask'].squeeze(0).cuda(),
            "rejected_input_ids": tokenized_rejected['input_ids'].squeeze(0).cuda(),
            "rejected_attention_mask": tokenized_rejected['attention_mask'].squeeze(0).cuda(),
        })

        orpo_dataset = Dataset.from_list(orpo_data)

(3) ORPO 설정

In [ ]:
from trl import ORPOConfig

orpo_config = ORPOConfig(
    output_dir='./orpo_output',
    per_device_train_batch_size=1,
    num_train_epochs=5,
    learning_rate=2e-6,
    gradient_accumulation_steps=4,
    logging_steps=50,
    fp16=False,
    bf16=True,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    warmup_steps=100,
    save_steps=500,
    save_total_limit=2
)

In [ ]:
from trl.trainer.utils import DPODataCollatorWithPadding

data_collator = DPODataCollatorWithPadding(
    pad_token_id=tokenizer.pad_token_id,
    label_pad_token_id=-100,
    is_encoder_decoder=False
)

In [ ]:
from trl import ORPOTrainer

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    train_dataset=orpo_dataset,
    data_collator=data_collator,
    processing_class=tokenizer
)

In [ ]:
import random

def generate_poem_final(num_samples=5):
    topics = ["바람", "비", "노을", "달빛", "안개", "사랑", "이별", "운명", "기다림", "후회", "추억", "시간", "청춘", "변화", "마지막 순간", "군중", "밤거리", "버스", "인생", "빌딩", "사람들", "거짓말", "욕망", "돈", "권력", "비밀", "죽음", "희망", "동물", "자연", "도시", "바다", "산", "하늘", "별", "꽃", "나무", "강", "바위", "흙", "눈", "빗방울", "눈물", "웃음"]
    result = []

    for _ in range(num_samples):
        topic = random.choice(topics)
        input_text = f'주제: {topic}\n시:'
        poem = generate_pipeline(
                                    input_text,
                                    max_new_tokens=100,
                                    temperature=0.8,
                                    top_p=0.9
                                )[0]['generated_text']
        result.append({"topic": topic, "poem": poem})

    return result

In [ ]:
generated_poem = generate_poem_final(num_samples=10)
generated_poem